# Advanced Indicators - Apple Stock 2023-2025

Testing advanced ML features for stock bottom detection.

**All indicators use backward-looking local extrema detection - NO LOOKAHEAD BIAS.**
Safe for ML features and real-time trading.

In [1]:
import sys
from pathlib import Path

import plotly.graph_objects as go
import yfinance as yf
from plotly.subplots import make_subplots

sys.path.insert(0, str(Path("../..").resolve()))

from indicators.advanced import (
    add_time_features,
    calculate_mean_reversion_signal,
    detect_bb_squeeze_breakdown,
    detect_exhaustion_sequence,
    detect_hidden_divergence,
    detect_multi_indicator_divergence,
    detect_panic_selling,
    detect_support_tests,
    detect_volume_exhaustion,
)

In [2]:
# Fetch Apple data
df = yf.download("AAPL", start="2023-01-01", end="2025-10-01", auto_adjust=True, progress=False)

# Handle MultiIndex columns from yfinance
if df.columns.nlevels == 2:
    df.columns = df.columns.get_level_values(0)

df.columns = df.columns.str.lower()
df = df.reset_index()
df.columns = df.columns.str.lower()

print(f"Data shape: {df.shape}")
df.head()

Data shape: (688, 6)


Price,date,close,high,low,open,volume
0,2023-01-03,123.330650,129.079567,122.443165,128.468194,112117500
1,2023-01-04,124.602707,126.870724,123.340509,125.125335,89113600
2,2023-01-05,123.281342,125.993097,123.024963,125.361998,80962700
3,2023-01-06,127.817360,128.478040,123.153145,124.257571,87754700
4,2023-01-09,128.339996,131.554669,128.083618,128.655553,70790800


## 1. Multi-Indicator Divergence

Detects bullish divergence: price makes lower low but indicators (RSI, MACD, Stochastic) make higher lows. 

Scores 0-3 based on number of diverging indicators.

Uses backward-looking local extrema (NO forward bias).

In [ ]:
df = detect_multi_indicator_divergence(df, lookback_window=8)

fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=("Price with Local Lows", "Multi-Indicator Divergence Score"),
)

fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "blue"}), row=1, col=1)

# Show local lows (backward-looking)
local_lows = df[df["LocalLow"] == 1]
fig.add_trace(
    go.Scatter(
        x=local_lows["date"],
        y=local_lows["close"],
        mode="markers",
        name="Local Low",
        marker={"color": "red", "size": 8},
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=df["date"], y=df["multi_divergence_score"], name="Divergence Score", fill="tozeroy", line={"color": "green"}
    ),
    row=2,
    col=1,
)

fig.update_layout(height=600, title_text="Multi-Indicator Divergence (Backward-Looking)")
fig.show()

print(f"Divergences detected: {(df['multi_divergence_score'] > 0).sum()}")
print(f"Max score: {df['multi_divergence_score'].max()}")

## 2. Volume Exhaustion

Price declining + volume declining = selling exhaustion. Sellers giving up despite price still falling. Normal selling has price down + volume up.

In [4]:
df = detect_volume_exhaustion(df)

fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=("Price", "Volume Exhaustion Binary", "Exhaustion Strength"),
)

fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close"), row=1, col=1)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["volume_exhaustion"], name="Exhaustion", fill="tozeroy", line={"color": "orange"}),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["exhaustion_strength"], name="Strength", fill="tozeroy", line={"color": "purple"}),
    row=3,
    col=1,
)

fig.update_layout(height=700, title_text="Volume Exhaustion")
fig.show()

## 3. Panic Selling

Extreme volume spike (>2x average) + extreme price drop (>2 std devs) = panic/capitulation. Often marks exact bottom.

In [5]:
df = detect_panic_selling(df)

fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=("Price", "Panic Selling Events", "Panic Severity"),
)

fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close"), row=1, col=1)
panic_events = df[df["panic_selling"] == 1]
fig.add_trace(
    go.Scatter(
        x=panic_events["date"],
        y=panic_events["close"],
        mode="markers",
        name="Panic Event",
        marker={"color": "red", "size": 10, "symbol": "x"},
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(x=df["date"], y=df["panic_selling"], name="Panic Binary", fill="tozeroy", line={"color": "red"}),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["panic_severity"], name="Severity", fill="tozeroy", line={"color": "darkred"}),
    row=3,
    col=1,
)

fig.update_layout(height=700, title_text="Panic Selling Detection")
fig.show()

## 4. Support Test Count

Counts how many times price has tested similar support levels (within 2% tolerance). More tests = stronger support = higher probability bounce.

In [6]:
df = detect_support_tests(df, tolerance=0.02, lookback_window=8)

fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, subplot_titles=("Price", "Support Test Count")
)

fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close"), row=1, col=1)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["support_test_count"], name="Test Count", fill="tozeroy", line={"color": "teal"}),
    row=2,
    col=1,
)

fig.update_layout(height=600, title_text="Support Level Testing (Backward-Looking)")
fig.show()

print(f"Max support tests: {df['support_test_count'].max()}")

Max support tests: 17


## 5. Exhaustion Sequence

Many consecutive down days BUT slowing drops = exhaustion = bottom near. Tracks consecutive negative days and whether selling is decelerating.

In [7]:
df = detect_exhaustion_sequence(df)

fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=("Price", "Consecutive Down Days", "Exhaustion Signal"),
)

fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close"), row=1, col=1)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["consecutive_down_days"], name="Down Days", fill="tozeroy", line={"color": "red"}),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["exhaustion_signal"], name="Signal", fill="tozeroy", line={"color": "green"}),
    row=3,
    col=1,
)

fig.update_layout(height=700, title_text="Exhaustion Sequence")
fig.show()

## 6. Hidden Bullish Divergence

HIGHER low in price (making progress) but LOWER low in RSI (underlying weakness). Different from regular divergence, indicates false breakdowns.

In [8]:
df = detect_hidden_divergence(df, lookback_window=8)

fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, subplot_titles=("Price", "Hidden Bullish Divergence")
)

fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close"), row=1, col=1)
hidden_div = df[df["hidden_bullish_divergence"] == 1]
fig.add_trace(
    go.Scatter(
        x=hidden_div["date"],
        y=hidden_div["close"],
        mode="markers",
        name="Hidden Div",
        marker={"color": "purple", "size": 8},
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=df["date"], y=df["hidden_bullish_divergence"], name="Signal", fill="tozeroy", line={"color": "purple"}
    ),
    row=2,
    col=1,
)

fig.update_layout(height=600, title_text="Hidden Bullish Divergence (Backward-Looking)")
fig.show()

print(f"Hidden divergences: {(df['hidden_bullish_divergence'] == 1).sum()}")

Hidden divergences: 5


## 7. Mean Reversion Signal

Statistical bottom detection: how far is price from long-term mean? Z-score < -2 means >2 standard deviations below mean (97.5th percentile).

In [ ]:
df = calculate_mean_reversion_signal(df)

fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=("Price vs Mean", "Z-Score", "Statistical Bottom Signal"),
)

fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close"), row=1, col=1)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["price_ma252"], name="252d MA", line={"color": "orange", "dash": "dash"}),
    row=1,
    col=1,
)

fig.add_trace(go.Scatter(x=df["date"], y=df["price_zscore"], name="Z-Score", line={"color": "blue"}), row=2, col=1)
fig.add_hline(y=-2, line_dash="dash", line_color="red", row=2, col=1)

fig.add_trace(
    go.Scatter(x=df["date"], y=df["statistical_bottom"], name="Bottom Signal", fill="tozeroy", line={"color": "red"}),
    row=3,
    col=1,
)

fig.update_layout(height=800, title_text="Mean Reversion Signal")
fig.show()

## 8. Bollinger Band Squeeze Breakdown

BB squeeze (low volatility, band width at 20-day low) followed by breakdown below lower band. Often precedes sharp reversal.

In [10]:
df = detect_bb_squeeze_breakdown(df)

fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=("Price with BB", "BB Width & Squeeze", "Squeeze Breakdown"),
)

fig.add_trace(
    go.Scatter(x=df["date"], y=df["bb_upper"], name="Upper BB", line={"color": "gray", "dash": "dash"}), row=1, col=1
)
fig.add_trace(go.Scatter(x=df["date"], y=df["bb_middle"], name="Middle BB", line={"color": "blue"}), row=1, col=1)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["bb_lower"], name="Lower BB", line={"color": "gray", "dash": "dash"}), row=1, col=1
)
fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black"}), row=1, col=1)

fig.add_trace(go.Scatter(x=df["date"], y=df["bb_width"], name="BB Width", line={"color": "orange"}), row=2, col=1)
fig.add_trace(
    go.Scatter(
        x=df["date"],
        y=df["bb_squeeze"] * df["bb_width"].max(),
        name="Squeeze",
        fill="tozeroy",
        line={"color": "yellow"},
        opacity=0.3,
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter(x=df["date"], y=df["squeeze_breakdown"], name="Breakdown", fill="tozeroy", line={"color": "red"}),
    row=3,
    col=1,
)

fig.update_layout(height=800, title_text="BB Squeeze Breakdown")
fig.show()

## 9. Time Features

Temporal patterns: day of week effects, days since last local low (cyclicality), month-end and quarter-end effects.

Uses backward-looking local extrema detection.

In [11]:
df = add_time_features(df, lookback_window=8)

fig = make_subplots(
    rows=4,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=("Price", "Day of Week", "Days Since Local Low", "Month/Quarter End"),
)

fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close"), row=1, col=1)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["day_of_week"], name="Day of Week", mode="markers", marker={"size": 3}), row=2, col=1
)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["days_since_last_low"], name="Days Since Low", line={"color": "green"}), row=3, col=1
)

fig.add_trace(
    go.Scatter(
        x=df["date"], y=df["is_month_end"], name="Month End", fill="tozeroy", line={"color": "blue"}, opacity=0.5
    ),
    row=4,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df["date"], y=df["is_quarter_end"], name="Quarter End", fill="tozeroy", line={"color": "red"}, opacity=0.5
    ),
    row=4,
    col=1,
)

fig.update_layout(height=900, title_text="Time-Based Features (Backward-Looking)")
fig.show()

print(f"Days since low range: {df['days_since_last_low'].min():.0f} - {df['days_since_last_low'].max():.0f}")

Days since low range: 0 - 57


## Combined View - All Advanced Features

In [ ]:
# Create comprehensive overview
fig = make_subplots(
    rows=5,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    subplot_titles=(
        "Price with Key Events",
        "Divergence & Support",
        "Volume Patterns",
        "Statistical Signals",
        "Volatility Patterns",
    ),
)

# Row 1: Price with events
fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black"}), row=1, col=1)
panic = df[df["panic_selling"] == 1]
fig.add_trace(
    go.Scatter(
        x=panic["date"],
        y=panic["close"],
        mode="markers",
        name="Panic",
        marker={"color": "red", "size": 8, "symbol": "x"},
    ),
    row=1,
    col=1,
)

# Show local lows instead of pivots
local_lows = df[df["LocalLow"] == 1]
fig.add_trace(
    go.Scatter(
        x=local_lows["date"],
        y=local_lows["close"],
        mode="markers",
        name="Local Low",
        marker={"color": "green", "size": 6},
    ),
    row=1,
    col=1,
)

# Row 2: Divergence & Support
fig.add_trace(
    go.Scatter(
        x=df["date"], y=df["multi_divergence_score"], name="Divergence", fill="tozeroy", line={"color": "purple"}
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["support_test_count"], name="Support Tests", line={"color": "teal", "dash": "dash"}),
    row=2,
    col=1,
)

# Row 3: Volume patterns
fig.add_trace(
    go.Scatter(
        x=df["date"], y=df["exhaustion_strength"], name="Vol Exhaustion", fill="tozeroy", line={"color": "orange"}
    ),
    row=3,
    col=1,
)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["panic_severity"], name="Panic Severity", line={"color": "red"}), row=3, col=1
)

# Row 4: Statistical
fig.add_trace(go.Scatter(x=df["date"], y=df["price_zscore"], name="Z-Score", line={"color": "blue"}), row=4, col=1)
fig.add_hline(y=-2, line_dash="dash", line_color="red", row=4, col=1)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["consecutive_down_days"], name="Down Days", line={"color": "brown", "dash": "dot"}),
    row=4,
    col=1,
)

# Row 5: Volatility
fig.add_trace(go.Scatter(x=df["date"], y=df["bb_width"], name="BB Width", line={"color": "orange"}), row=5, col=1)
fig.add_trace(
    go.Scatter(
        x=df["date"],
        y=df["squeeze_breakdown"] * df["bb_width"].max(),
        name="Squeeze Breakdown",
        fill="tozeroy",
        line={"color": "red"},
        opacity=0.5,
    ),
    row=5,
    col=1,
)

fig.update_layout(height=1200, title_text="Advanced Features - Complete Overview (Backward-Looking)")
fig.show()

print("\n=== Feature Summary ===")
print(f"Local lows detected: {(df['LocalLow'] == 1).sum()}")
print(f"Multi-indicator divergences: {(df['multi_divergence_score'] > 0).sum()}")
print(f"Hidden divergences: {(df['hidden_bullish_divergence'] == 1).sum()}")
print(f"Panic selling events: {(df['panic_selling'] == 1).sum()}")
print(f"Max support tests: {df['support_test_count'].max()}")
print("\nAll features use backward-looking detection - safe for ML!")